# Finding short superpermutations for n=4

In [1]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import SubprocVecEnv
from GymPermutationsEnv import GymPermutationEnv

In [2]:
import sys
from typing import Any, Dict, Tuple, Union
import mlflow
import numpy as np

from stable_baselines3.common.logger import HumanOutputFormat, KVWriter, Logger

class MLflowOutputFormat(KVWriter):
    """
    Dumps key/value pairs into MLflow's numeric format.
    """

    def write(
        self,
        key_values: Dict[str, Any],
        key_excluded: Dict[str, Union[str, Tuple[str, ...]]],
        step: int = 0,
    ) -> None:

        for (key, value), (_, excluded) in zip(
            sorted(key_values.items()), sorted(key_excluded.items())
        ):

            if excluded is not None and "mlflow" in excluded:
                continue

            if isinstance(value, np.ScalarType):
                if not isinstance(value, str):
                    mlflow.log_metric(key, value, step)


loggers = Logger(
    folder=None,
    output_formats=[HumanOutputFormat(sys.stdout), MLflowOutputFormat()],
)
mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

In [3]:
from stable_baselines3.common.callbacks import BaseCallback, CallbackList

class PermutationLogCallback(BaseCallback):
    def __init__(self, verbose=0):
        super().__init__(verbose)
        self.best_permutation = []
        self.best_permutation_length=-1

    def _on_step(self) -> bool:
        infos = self.locals.get("infos", [])
        for _info in infos:
            if "superpermutation" in _info and "superpermutation_length" in _info:
                if self.best_permutation_length==-1 or (_info["superpermutation_length"] < self.best_permutation_length):
                    self.best_permutation_length = _info["superpermutation_length"]
                    self.best_permutation = " ".join([str(i) for i in _info["superpermutation"]])
                self.logger.record("superpermutation/best_superpermutation_length", self.best_permutation_length)
                self.logger.record("superpermutation/best_superpermutation", self.best_permutation)
        return True


In [4]:
# Train the agent
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
import permutation_utils

alphabet_size=4
vec_env = make_vec_env(GymPermutationEnv, n_envs=16, env_kwargs=dict(alphabet_size=alphabet_size), vec_env_cls=SubprocVecEnv)
eval_env = make_vec_env(GymPermutationEnv, env_kwargs=dict(alphabet_size=alphabet_size), vec_env_cls=SubprocVecEnv)

max_reward_unscaled = permutation_utils.get_max_possible_reward(4, 33)
max_reward_scaled = max_reward_unscaled/alphabet_size

# Set up hyperparameters (mostly defaults)
hp_policy_type = "MlpPolicy"
hp_learning_rate = 3e-4
hp_clip_range = 0.2
hp_batch_size = 64
hp_n_steps=2048
hp_seed = 42 # Not really a hyperparameter, unless we're extremely unlucky...
hp_training_timesteps = 10_000_000


callback_on_best = StopTrainingOnRewardThreshold(reward_threshold=max_reward_scaled, verbose=1)
eval_callback = EvalCallback(eval_env, callback_on_new_best=callback_on_best, verbose=1, n_eval_episodes=20, deterministic=False)
permutation_metrics_callback = PermutationLogCallback()
callback_list = CallbackList([eval_callback, permutation_metrics_callback])

with mlflow.start_run():
    mlflow.log_param("alphabet_size", alphabet_size)

    mlflow.log_param("policy_type", hp_policy_type)
    mlflow.log_param("learning_rate", hp_learning_rate)
    mlflow.log_param("clip_range", hp_clip_range)
    mlflow.log_param("batch_size", hp_batch_size)
    mlflow.log_param("n_steps", hp_n_steps)
    mlflow.log_param("seed", hp_seed)
    mlflow.log_param("training_timesteps", hp_training_timesteps)

    model = PPO(hp_policy_type,
                vec_env,
                verbose=1,
                batch_size=hp_batch_size,
                clip_range=hp_clip_range,
                seed=hp_seed,
                n_steps=hp_n_steps,
                learning_rate=hp_learning_rate)
    # Set custom logger
    model.set_logger(loggers)
    model.learn(int(hp_training_timesteps), callback=callback_list)

Using cpu device
--------------------------------------------------------------------------
| rollout/                        |                                      |
|    ep_len_mean                  | 34                                   |
|    ep_rew_mean                  | -11.2                                |
| superpermutation/               |                                      |
|    best_superpermutation        | 4 2 1 3 1 2 3 4 1 2 3 1 2 4 3 1 2... |
|    best_superpermutation_length | 40                                   |
| time/                           |                                      |
|    fps                          | 6912                                 |
|    iterations                   | 1                                    |
|    time_elapsed                 | 4                                    |
|    total_timesteps              | 32768                                |
--------------------------------------------------------------------------
--------

KeyboardInterrupt: 

The model actually finds the optimal solution fairly quickly but it takes a while for early stopping to kick in since it doesn't find it reliably (we store it anyway so the training can be cancelled manually once we see that the solution is good enough)

In [5]:
print(permutation_metrics_callback.best_permutation_length)
print(permutation_metrics_callback.best_permutation)

33
4 3 2 1 4 3 2 4 1 3 2 4 3 1 2 4 3 4 2 1 3 4 2 3 1 4 2 3 4 1 2 3 4


### The RL agent found the following superpermutation:
4 3 2 1 4 3 2 4 1 3 2 4 3 1 2 4 3 4 2 1 3 4 2 3 1 4 2 3 4 1 2 3 4

We can check if it's the same (up to relabelling) as the shortest superpermutation shown on Wikipedia:

In [7]:
import permutation_utils
found = [int(i) for i in "4 3 2 1 4 3 2 4 1 3 2 4 3 1 2 4 3 4 2 1 3 4 2 3 1 4 2 3 4 1 2 3 4".split(" ")]
shortest_superpermutation = [int(i) for i in "123412314231243121342132413214321"]
relabellings_list = permutation_utils.get_possible_relabellings(shortest_superpermutation, [1,2,3,4])
print(found in relabellings_list)

True


### Calculating the maximum possible reward for an episode

Assume that n is the size of the alphabet. Let us call the superpermutation formed by appending every possible permutation a "naive superpermutation".

Since there are n! possible permutations, each having length n and every one has to be included, the naive superpermutation has length n!*n.

The rewards in the environment are formed in such a way that the model gets 1 point for each character "saved" compared to this naive superpermutation. That is, if merging two permutations that have 2 overlapping characters, we get 2 points. If by merging we also added another permutation (other than the two we merged), we additionally reward the model with n points.

Thus the cumulative reward for an episode is (aside from penalties for picking already added permutations) equal to (n!*n)-(length of the superpermutation created by the model).

So the length of the superpermutation created by the model is at most (n!*n)-(sum of cumulative rewards for the episode)

For n=4, this is (4!*4)-(sum of cumulative rewards for the episode).

33 = 96-reward-->reward=63